In [ ]:
!pip install pyyaml==5.1

In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

In [ ]:
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

In [ ]:
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

# Commented out IPython magic to ensure Python compatibility.
#Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import matplotlib.pyplot as plt
# %matplotlib inline

from detectron2.structures import BoxMode

In [ ]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        #record["height"] =3000
        #record["width"] = 2000
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
classes = ['rod']


#File Path needs to be chnged according to the firectory where it is saved
data_path = '/content/drive/MyDrive/abc/abc/scoliosis 2000 3000/'

for d in ["train", "test24"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_train")

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 200
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)

trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.DATASETS.TEST = ("rod_detection", )
predictor = DefaultPredictor(cfg)

test_dataset_dicts = get_data_dicts(data_path+'test24', classes)

outputs = list()
images = list()
for d in test_dataset_dicts:    
    img = cv2.imread(d["file_name"])
    images.append(img)
    o = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=microcontroller_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
    )
    outputs.append(o)
    v = v.draw_instance_predictions(o["instances"].to("cpu"))
    plt.figure(figsize = (28,20))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
from google.colab.patches import cv2_imshow

from PIL import Image
import PIL
for i, obj in enumerate(outputs): ### [(0, p1]), (1, p2)]
  masks = np.asarray(obj['instances'].pred_masks.to("cpu"))
  item_mask = masks[0]
  # Get the true bounding box of the mask (not the same as the bbox prediction)
  segmentation = np.where(item_mask == True)
  x_min = int(np.min(segmentation[1]))
  x_max = int(np.max(segmentation[1]))
  y_min = int(np.min(segmentation[0]))
  y_max = int(np.max(segmentation[0]))
  print(x_min, x_max, y_min, y_max)

  # Create a cropped image from just the portion of the image we want
  cropped = Image.fromarray(images[i][y_min:y_max, x_min:x_max, :], mode='RGB')
  cropped.save("/content/output/"+str(i+1)+".jpg")

  plt.figure(figsize = (12,8))
  plt.imshow(cropped)
  plt.show()

In [ ]:
"""from PIL import Image

# Get the masks
masks = np.asarray(outputs["instances"].pred_masks.to("cpu"))

# Pick an item to mask
item_mask = masks[0]

# Get the true bounding box of the mask (not the same as the bbox prediction)
segmentation = np.where(item_mask == True)
x_min = int(np.min(segmentation[1]))
x_max = int(np.max(segmentation[1]))
y_min = int(np.min(segmentation[0]))
y_max = int(np.max(segmentation[0]))
print(x_min, x_max, y_min, y_max)

# Create a cropped image from just the portion of the image we want
cropped = Image.fromarray(img[y_min:y_max, x_min:x_max, :], mode='RGB')
plt.imshow(cropped)

#J = cv2.imwrite('/content/drive/MyDrive/abc/poly/'+ '.jpg',cropped)

# Create a PIL image out of the mask
mask = Image.fromarray((item_mask * 255).astype('uint8'))
#plt.imshow(mask)


# Crop the mask to match the cropped image
cropped_mask = mask.crop((x_min, y_min, x_max, y_max))
#plt.imshow(cropped_mask)
"""
"""
lightgrayimage = 200 * np.ones(shape=[512, 512, 3], dtype=np.uint8)
cv2_imshow(lightgrayimage)

# Load in a background image and choose a paste position
background = Image.fromarray(img, mode='RGB')
paste_position = (1000,1500)

# Create a new foreground image as large as the composite and paste the cropped image on top
new_fg_image = Image.new('RGB', background.size)
new_fg_image.paste(cropped, paste_position)

# Create a new alpha mask as large as the composite and paste the cropped mask
new_alpha_mask = Image.new('L', background.size, color = 0)
new_alpha_mask.paste(cropped_mask, paste_position)

# Compose the foreground and background using the alpha mask
composite = Image.composite(new_fg_image, background, new_alpha_mask)

# Display the image
cv2_imshow(np.array(composite))

"""
